# Lab 05: Working with Microsoft Graph REST API

This lab will guide you through using the Microsoft Graph REST API directly from PowerShell using the Microsoft Graph PowerShell SDK. You will learn how to make direct REST calls, work with JSON batching, and understand how to use access tokens with Invoke-RestMethod.

## Required Permissions

The permissions needed will vary based on the API endpoints you're accessing. For this lab, we'll be working with:

- **User.Read** - For accessing the `/me` endpoint
- **AuditLog.Read.All** - For accessing sign-in logs
- **Group.Read.All** - For accessing group information

## Authentication with Microsoft Graph

Before you can perform any operations, you need to authenticate to Microsoft Graph.

In [ ]:
# Required modules
# Import-Module Microsoft.Graph.Users
# Import-Module Microsoft.Graph.Groups
# Import-Module Microsoft.Graph.Applications

# Connect to Microsoft Graph with the required permissions
Connect-MgGraph -Scopes "User.Read", "AuditLog.Read.All", "Group.Read.All"

# Verify the connection and permissions
Get-MgContext

## Section 1: Overview of REST API Calls with Microsoft Graph PowerShell SDK

The Microsoft Graph PowerShell SDK provides cmdlets that wrap the Graph API endpoints. However, sometimes you need to directly access the REST API when:

1. A specific API doesn't have a dedicated cmdlet yet
2. You need more control over the request/response
3. You're working with advanced query parameters or beta endpoints
4. You want to batch multiple requests for efficiency

The primary tool for this is the `Invoke-MgGraphRequest` cmdlet. Let's look at its syntax:

In [ ]:
# View the syntax of Invoke-MgGraphRequest cmdlet
Get-Command Invoke-MgGraphRequest -Syntax

### Understanding Graph API URIs

Microsoft Graph URLs follow this structure:

```
https://graph.microsoft.com/{version}/{resource}?{query-parameters}
```

Where:
- **version**: Either `v1.0` (stable) or `beta` (preview)
- **resource**: The API resource path (e.g., `/users`, `/me`, `/groups`)
- **query-parameters**: Optional OData parameters like `$select`, `$filter`, `$expand`, etc.

For example, to get information about the current user:
```
https://graph.microsoft.com/v1.0/me
```

Or to get a specific user by ID:
```
https://graph.microsoft.com/v1.0/users/{id}
```

### Finding the Right REST URI

To find the right API endpoints and parameters, you can use:

1. **Graph Explorer**: https://developer.microsoft.com/en-us/graph/graph-explorer (or https://aka.ms/ge)
2. **Microsoft Graph documentation**: https://docs.microsoft.com/en-us/graph/api/overview
3. **Graph X-Ray**: A browser extension that shows Graph API calls made by the Azure and Entra portals

## Section 2: Task with Invoke-MgGraphRequest

Let's practice using `Invoke-MgGraphRequest` to make direct API calls. We'll start with simple calls and then explore different output types.

In [ ]:
# Example 1: Get current user information from v1.0 endpoint
Invoke-MgGraphRequest -Method GET -Uri "https://graph.microsoft.com/v1.0/me"

In [ ]:
# Example 2: Get more detailed user information from beta endpoint
Invoke-MgGraphRequest -Method GET -Uri "https://graph.microsoft.com/beta/me"

In [ ]:
# Example 3: Select specific properties with $select
Invoke-MgGraphRequest -Method GET -Uri "https://graph.microsoft.com/v1.0/me?$select=displayName,mail,jobTitle,department"

### Exploring Different Output Types

`Invoke-MgGraphRequest` supports different output formats with the `-OutputType` parameter:

- **HashTable** (default): Returns a PowerShell hashtable
- **HttpResponseMessage**: Returns the full HTTP response including headers
- **Json**: Returns raw JSON string
- **PSObject**: Returns PowerShell objects

In [ ]:
# Try different output types
# Default HashTable output
Invoke-MgGraphRequest -Method GET -Uri "/beta/me" -OutputType HashTable

In [ ]:
# HttpResponseMessage output (includes headers, status, etc.)
$response = Invoke-MgGraphRequest -Method GET -Uri "/beta/me" -OutputType HttpResponseMessage
# View response properties
$response | Get-Member
# View status code
$response.StatusCode

In [ ]:
# JSON output
$jsonResponse = Invoke-MgGraphRequest -Method GET -Uri "/beta/me" -OutputType Json
# Convert from JSON to view it nicely
$jsonResponse | ConvertFrom-Json | Format-List

In [ ]:
# PSObject output
$psObject = Invoke-MgGraphRequest -Method GET -Uri "/beta/me" -OutputType PSObject
$psObject | Get-Member
$psObject | Format-List

### Exercise: Working with Groups API

Now, try to use `Invoke-MgGraphRequest` to retrieve information about a group and its members. First, let's list some groups to get an ID to work with.

In [ ]:
# List groups to find an ID to work with
$groups = Invoke-MgGraphRequest -Method GET -Uri 'https://graph.microsoft.com/v1.0/groups?$select=id,displayName,mailEnabled,securityEnabled&$top=5' -OutputType PSObject
$groups.value | Format-Table id, displayName, mailEnabled, securityEnabled

# Store the first group ID for later use (choose a different one if needed)
$groupId = $groups.value[0].id

In [ ]:
# Get detailed information about this group
$groupInfo = Invoke-MgGraphRequest -Method GET -Uri "https://graph.microsoft.com/v1.0/groups/$($groupId)?`$select=id,displayName,description,createdDateTime,mail,mailEnabled,mailNickname,securityEnabled" -OutputType PSObject
$groupInfo | Format-List

In [ ]:
# Get members of this group
$groupMembers = Invoke-MgGraphRequest -Method GET -Uri "https://graph.microsoft.com/v1.0/groups/$($groupId)/members?`$select=id,displayName,userPrincipalName,mail" -OutputType PSObject
$groupMembers.value | Format-Table displayName, userPrincipalName, mail

## Section 3: JSON Batching

Microsoft Graph supports JSON batching, which allows you to combine multiple requests into a single HTTP request. This is useful for reducing network overhead and improving performance.

Let's create a batch request to fetch different types of sign-in events.

In [ ]:
# Define dates for the query (adjust as needed)
$startDate = (Get-Date).AddDays(-30).ToString("yyyy-MM-ddT00:00:00Z")
$endDate = Get-Date -Format "yyyy-MM-ddT23:59:59Z"

# Create a batch request to fetch different sign-in event types
$batchRequest = @{
    requests = @(
        @{
            id = "interactive"
            method = "GET"
            url = "/auditLogs/signIns?`$filter=createdDateTime ge $startDate and createdDateTime le $endDate&`$top=5"
        },
        @{
            id = "nonInteractive"
            method = "GET"
            url = "/auditLogs/signIns?`$filter=createdDateTime ge $startDate and createdDateTime le $endDate and signInEventTypes/any(t:t eq 'nonInteractiveUser')&`$top=5"
        },
        @{
            id = "servicePrincipal"
            method = "GET"
            url = "/auditLogs/signIns?`$filter=createdDateTime ge $startDate and createdDateTime le $endDate and signInEventTypes/any(t:t eq 'servicePrincipal')&`$top=5"
        },
        @{
            id = "managedIdentity"
            method = "GET"
            url = "/auditLogs/signIns?`$filter=createdDateTime ge $startDate and createdDateTime le $endDate and signInEventTypes/any(t:t eq 'managedIdentity')&`$top=5"
        }
    )
}

In [ ]:
# Send the batch request to Microsoft Graph
$batchResponse = Invoke-MgGraphRequest -Uri 'https://graph.microsoft.com/beta/$batch' -Method POST -Headers @{
    'Content-Type' = 'application/json'
} -Body ($batchRequest | ConvertTo-Json -Depth 4)

# Display response overview
$batchResponse.responses | Select-Object id, status, @{Name='Count';Expression={$_.body['value'].Count}} | Format-Table

In [ ]:
# Convert responses to PSCustomObjects for easier processing
$Responses = $batchResponse.responses | ForEach-Object {
    [pscustomobject]$_
}

# View service principal sign-ins (first one)
$Responses | Where-Object id -eq 'servicePrincipal' | Select-Object -ExpandProperty body | 
    Select-Object -ExpandProperty value | Select-Object -First 1 | 
    Select-Object createdDateTime, appDisplayName, appId, ipAddress, status

In [ ]:
# Try with PSObject output type
$batchResponse = Invoke-MgGraphRequest -Uri 'https://graph.microsoft.com/beta/$batch' -Method POST -Headers @{
    'Content-Type' = 'application/json'
} -Body ($batchRequest | ConvertTo-Json -Depth 4) -OutputType PSObject

# View interactive sign-ins
$interactiveSignIns = $batchResponse.responses | Where-Object id -eq 'interactive' | 
    Select-Object -ExpandProperty body | Select-Object -ExpandProperty value

$interactiveSignIns | Select-Object createdDateTime, userDisplayName, userPrincipalName, appDisplayName, status | Format-Table

### Exercise: Create Your Own Batch Request

Now, create your own batch request that combines different operations. For example, try to get:
1. Your user profile
2. Your recent emails (if you have Mail.Read permission)
3. A list of groups you belong to

In [ ]:
# Create your own batch request here
$myBatchRequest = @{
    requests = @(
        @{
            id = "profile"
            method = "GET"
            url = '/me?$select=displayName,jobTitle,mail,officeLocation'
        },
        @{
            id = "memberOf"
            method = "GET"
            url = '/me/memberOf?$select=id,displayName&$top=5'
        }
        # Add more requests as needed
    )
}

# Send your batch request
$myBatchResponse = Invoke-MgGraphRequest -Uri 'https://graph.microsoft.com/v1.0/$batch' -Method POST -Body ($myBatchRequest | ConvertTo-Json -Depth 4) -OutputType PSObject -Headers @{
    'Content-Type' = 'application/json'
}

# Process the responses
$myBatchResponse.responses | ForEach-Object {
    Write-Host "ID: $($_.id) - Status: $($_.status)" -ForegroundColor Cyan
    if ($_.body) {
        if ($_.id -eq "profile") {
            $_.body | Format-List
        } elseif ($_.id -eq "memberOf") {
            $_.body.value | Format-Table displayName
        }
    }
}

## Section 4: Bonus Task - Working with Access Tokens and Invoke-RestMethod

Sometimes, you might want to extract the access token from your Microsoft Graph session and use it with `Invoke-RestMethod`. This is useful when:

1. You need more control over the HTTP request
2. You want to use the same token with other REST APIs
3. You're building a script that will run in environments without the Microsoft Graph PowerShell SDK

In [ ]:
# First, get an access token by making any request with the HttpResponseMessage output type
$data = Invoke-MgGraphRequest -Uri "https://graph.microsoft.com/v1.0/me" -Method GET -OutputType HttpResponseMessage

# Extract the token from the Authorization header
$token = $data.RequestMessage.Headers.Authorization.Parameter

# Show first few characters of the token (never display the full token in logs or outputs)
"Token: $($token.Substring(0, 15))..." 

In [ ]:
# Now use this token with Invoke-RestMethod
$headers = @{ Authorization = "Bearer $token" }

# Make a request to get current user information
$response = Invoke-RestMethod -Uri "https://graph.microsoft.com/v1.0/me" -Headers $headers -Method GET
$response | Format-List

In [ ]:
# Try another request to list your groups
$response = Invoke-RestMethod -Uri "https://graph.microsoft.com/v1.0/me/memberOf?`$select=displayName,mailEnabled,securityEnabled&`$top=5" -Headers $headers -Method GET
$response.value | Format-Table displayName, mailEnabled, securityEnabled

### Examine Token Information (Optional)

If you want to examine the claims in the token, you can decode it using the following code:

In [ ]:
function Parse-JWTtoken {
    param([string]$token)
    
    # Split the token parts
    $tokenParts = $token.Split('.')
    
    # Base64 decode and convert to JSON
    $decodedToken = [System.Text.Encoding]::UTF8.GetString([System.Convert]::FromBase64String(($tokenParts[1].Replace('-', '+').Replace('_', '/') + ('=' * (4 - $tokenParts[1].Length % 4)))))  
    
    # Convert from JSON
    return $decodedToken | ConvertFrom-Json
}
function Decode-JwtToken {
    param (
        [string]$token
    )

    $parts = $token -split '\.'
    $payload = $parts[1]
    $padding = $payload.Length % 4
    if ($padding -ne 0) {
        $payload += '=' * (4 - $padding)
    }
    $decodedPayload = [System.Text.Encoding]::UTF8.GetString([Convert]::FromBase64String($payload))
    $jsonPayload = $decodedPayload | ConvertFrom-Json
    return $jsonPayload
}

# Parse the token
$decodedToken = Decode-JWTtoken -token $token

# Display key claims
$decodedToken | Select-Object -Property name, upn, oid, roles, scp | Format-List

## Conclusion

In this lab, you've learned how to:

1. Make direct REST API calls to Microsoft Graph using `Invoke-MgGraphRequest`
2. Work with different output formats to process responses
3. Use JSON batching to combine multiple requests
4. Extract and use access tokens with `Invoke-RestMethod`

These skills will help you work with Microsoft Graph more efficiently and access APIs that might not be covered by dedicated cmdlets yet.